In [ ]:


from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import matplotlib_inline
import re
import math
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import nltk
from nltk.corpus import stopwords
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import Manager



In [ ]:
df = pd.read_csv("995,000_rows.csv", dtype={0: str, 1: str}) 
#df = pd.read_csv("news_sample.csv", dtype={0: str, 1: str}) 
df = df.drop(['keywords', 'tags', 'authors', 'meta_description', 'summary'], axis=1)
df = df[df['type'] != '2018-02-10 13:43:39.521661']
df = df[df['type'] != 'nan']
df = df[df['type'].notna()]
df = df[df['content'].str.strip() != '']

In [ ]:
def clean_text(text):
    """Redacts URLs, dates, email addresses and numbers in a given text input, as well as converting text to lower case and removing tabs, newlines, and spaces following other spaces"""
    text = str(text)
    date_exp =  {
                "year_mm_dd" : re.compile(r'[^\d]{1}([0-9]{2,4})[\s\/\.\-\\]?([0-1]{1}[0-9]{1})[\s\/\.\-\\]?([0-3]{1}[0-9]{1})\s?([\d]{2}:[\d]{2}:[\d]{2}\.[\d]{6})?', re.MULTILINE),   
                "dd_mm_year" : re.compile(r'[^\d]{1}([0-3]{1}[0-9]{1})[\s\/\.\-\\]?([0-1]{1}[0-9]{1})[\s\/\.\-\\]?([0-9]{2,4})\s?([\d]{2}:[\d]{2}:[\d]{2}\.[\d]{6})?', re.MULTILINE),
                "mm_dd_year" : re.compile(r'[^\d]{1}([0-1]{1}[0-9]{1})[\s\/\.\-\\]?([0-3]{1}[0-9]{1})[\s\/\.\-\\]?([0-9]{2,4})\s?([\d]{2}:[\d]{2}:[\d]{2}\.[\d]{6})?', re.MULTILINE),
                "year_mm_dd_time" : re.compile(r'[^\d]{1}([0-9]{2,4})[\s\/\.\-\\]?([0-1]{1}[0-9]{1})[\s\/\.\-\\]?([0-3]{1}[0-9]{1})\s?([\d]{2}:[\d]{2}:[\d]{2}\.[\d]{6})?', re.MULTILINE),
                "dd_mm_year_time" : re.compile(r'[^\d]{1}([0-3]{1}[0-9]{1})[\s\/\.\-\\]?([0-1]{1}[0-9]{1})[\s\/\.\-\\]?([0-9]{2,4})\s?([\d]{2}:[\d]{2}:[\d]{2}\.[\d]{6})?', re.MULTILINE),
                "mm_dd_year_time" : re.compile(r'[^\d]{1}([0-1]{1}[0-9]{1})[\s\/\.\-\\]?([0-3]{1}[0-9]{1})[\s\/\.\-\\]?([0-9]{2,4})\s?([\d]{2}:[\d]{2}:[\d]{2}\.[\d]{6})?', re.MULTILINE),
                "year_mm_dd_hh_mm" : re.compile(r'[^\d]{1}([0-9]{2,4})[\s\/\.\-\\]?([0-1]{1}[0-9]{1})[\s\/\.\-\\]?([0-3]{1}[0-9]{1})\s?([\d]{2}:[\d]{2})', re.MULTILINE),
                "dd_mm_year_hh_mm" : re.compile(r'[^\d]{1}([0-3]{1}[0-9]{1})[\s\/\.\-\\]?([0-1]{1}[0-9]{1})[\s\/\.\-\\]?([0-9]{2,4})\s?([\d]{2}:[\d]{2})', re.MULTILINE),
                "mm_dd_year_hh_mm" : re.compile(r'[^\d]{1}([0-1]{1}[0-9]{1})[\s\/\.\-\\]?([0-3]{1}[0-9]{1})[\s\/\.\-\\]?([0-9]{2,4})\s?([\d]{2}:[\d]{2})', re.MULTILINE),
                "year_mm_dd_hh_mm_ss" : re.compile(r'[^\d]{1}([0-9]{2,4})[\s\/\.\-\\]?([0-1]{1}[0-9]{1})[\s\/\.\-\\]?([0-3]{1}[0-9]{1})\s?([\d]{2}:[\d]{2}:[\d]{2})', re.MULTILINE),
                "dd_mm_year_hh_mm_ss" : re.compile(r'[^\d]{1}([0-3]{1}[0-9]{1})[\s\/\.\-\\]?([0-1]{1}[0-9]{1})[\s\/\.\-\\]?([0-9]{2,4})\s?([\d]{2}:[\d]{2}:[\d]{2})', re.MULTILINE),
                "mm_dd_year_hh_mm_ss" : re.compile(r'[^\d]{1}([0-1]{1}[0-9]{1})[\s\/\.\-\\]?([0-3]{1}[0-9]{1})[\s\/\.\-\\]?([0-9]{2,4})\s?([\d]{2}:[\d]{2}:[\d]{2})', re.MULTILINE),
                }
    num_exp = re.compile('[0-9]+[,.]?[0-9]*', re.MULTILINE)
    num2_exp = re.compile(r'([0-9]+)((st)?(nd)?(rd)?(th)?(st)?){1}')
    url_exp = re.compile(r'((h{1}t{2}p{1}s?\:{1}\/{2})|(w{3}\.{1})){0,2}[^,\s]*\.[a-zA-Z]{2,}[^,\s]*', re.MULTILINE)
    email_exp = re.compile(r'[^,\s\/]*@{1}[^,\s\/]*\.[a-zA-Z]{2,3}', re.MULTILINE)
    space_exp = re.compile(r'([\s]{2,})|[\t]|[\n]+', re.MULTILINE)
    punctuation_exp = re.compile(r'[^\w\s]', re.MULTILINE)

    text = text.lower() #Convert to lower case before inserting capitalized placeholders
    for exp in date_exp.values():
        text = exp.sub('DATETOKEN', text) #Replace dates before numbers
    text = num_exp.sub('NUMTOKEN', text)
    text = url_exp.sub('URLTOKEN', text)
    text = email_exp.sub('EMAILTOKEN', text)
    text = space_exp.sub(' ', text)
    text = punctuation_exp.sub(' ', text)
    return text

def clean_text_series(series):
    return series.apply(clean_text)

def tokenize_and_stem_series(series):
    stop_words = set(stopwords.words('english'))
    stemmer = nltk.PorterStemmer()
    return series.apply(lambda x: [stemmer.stem(token) for token in nltk.word_tokenize(x) if token not in stop_words])

n = 50000
list_df = np.array_split(df, math.ceil(len(df) / n))
del df

j = 1
for df in list_df:
    print(f"Processing chunk {j} of {len(list_df)}:")
    # Check if chunk parquet already exists
    try:
        pq.read_table(f"chunk_{j}.parquet").to_pandas()
        print(f"Chunk {j} already processed!", flush=True)
        df = pd.read_parquet(f"chunk_{j}.parquet")
        j += 1
        continue
    except:
        pass

    print("Changing column types...", flush=True)
    df['type'] = df['type'].astype('category')
    df['domain'] = df['domain'].astype('category')
    print("Changing datetime columns...", end="\r", flush=True)
    df['scraped_at'] = pd.to_datetime(df['scraped_at'], format='mixed', utc=True)
    df['inserted_at'] = pd.to_datetime(df['inserted_at'], format='mixed')
    df['updated_at'] = pd.to_datetime(df['updated_at'], format='mixed')

    print("Cleaning content...", flush=True)
    df['content'] = clean_text_series(df['content'])
    print("Tokenizing, stemming and removing stopwords from content...", flush=True)
    df['content'] = tokenize_and_stem_series(df['content'])
    print("Calculating features...", end="\r", flush=True)
    df['length'] = df['content'].apply(len)
    df['distinct_words'] = df['content'].apply(set)
    df['length_distinct_words'] = df['distinct_words'].apply(len)

    print("Saving chunk...", flush=True)
    df.to_parquet(f"chunk_{j}.parquet")
    print(f"Chunk {j} done!", flush=True)

    j += 1

df = pd.concat([pd.read_parquet(f"chunk_{j}.parquet") for j in range(1, len(list_df) + 1)])
del list_df

In [ ]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.bar(df["type"].value_counts().index, df["type"].value_counts().values)
ax1.set_title(f'Article types')
ax1.set_xlabel('Types')
plt.xticks(rotation=45)
ax1.set_ylabel('# of articles')
fig1.savefig(f'data\\articles_of_each_type.png')
fig1.show()

In [ ]:
for type in df['type'].unique():
    name = f'{type}_df'
    globals()[name] = df[df['type'] == type]

    print(f'Articles of type {type}: {len(globals()[name])}')

    print(f'{type} mean length: {globals()[name]["length"].mean()}')
    print(f'{type} median length: {globals()[name]["length"].median()}')

    print(f'{type} mean distinct words: {globals()[name]["length_distinct_words"].mean()}')
    print(f'{type} median distinct words: {globals()[name]["length_distinct_words"].median()}')

    print('\n')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Calculate global min and max for length and length_distinct_words
global_min_length = df['length'].min()
global_max_length = df['length'].max()
global_min_length_distinct_words = df['length_distinct_words'].min()
global_max_length_distinct_words = df['length_distinct_words'].max()

# Main plot with all data points
fig1, ax1 = plt.subplots(figsize=(10, 10))

# Use a color map to automatically assign colors
color_map = plt.get_cmap('tab10')

# Plot each type with a different color and set the size of the dots
dot_size = 1  # Adjust this value to change the size of the dots
for i, article_type in enumerate(df['type'].unique()):
    type_df = df[df['type'] == article_type]
    ax1.scatter(type_df['length_distinct_words'], type_df['length'], color=color_map(i), label=article_type, s=dot_size)

ax1.set_title('Article length vs number of distinct words')
ax1.set_xlabel('Number of distinct words')
ax1.set_ylabel('Article length')
ax1.legend()
ax1.set_xlim(global_min_length_distinct_words, global_max_length_distinct_words)
ax1.set_ylim(global_min_length, global_max_length)

# Create a new figure for the grid of subplots
num_types = len(df['type'].unique())
ncols = 2
nrows = (num_types + 1) // ncols  # Calculate the number of rows needed

fig2, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 5 * nrows))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each type in a separate subplot
for i, article_type in enumerate(df['type'].unique()):
    type_df = df[df['type'] == article_type]
    ax = axes[i]
    ax.scatter(type_df['length_distinct_words'], type_df['length'], color=color_map(i), label=article_type, s=dot_size)
    ax.set_title(f'{article_type}')
    ax.set_xlabel('Number of distinct words')
    ax.set_ylabel('Article length')
    ax.legend()
    ax.set_xlim(global_min_length_distinct_words, global_max_length_distinct_words)
    ax.set_ylim(global_min_length, global_max_length)

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig2.delaxes(axes[j])

# Adjust layout to prevent overlap
plt.tight_layout()

# Save the figures
fig1.savefig('data/article_length_vs_distinct_words.png')
fig2.savefig('data/article_length_vs_distinct_words_by_type.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

# Create the directory if it does not exist
output_dir = 'data/article_length_distributions'
os.makedirs(output_dir, exist_ok=True)

# Calculate global min and max for length and log(length)
global_min_length = df['length'].min()
global_max_length = df['length'].max()
global_min_log_length = np.log(df['length']).min()
global_max_log_length = np.log(df['length']).max()

for article_type in df['type'].unique():
    fig1 = plt.figure(figsize=(10, 5))
    # Adding two subplots side by side
    ax1 = fig1.add_subplot(1, 2, 1)
    ax2 = fig1.add_subplot(1, 2, 2)
    
    type_df = df[df['type'] == article_type]
    
    # The first subplot shows the distribution of the length of the articles
    ax1.hist(type_df['length'], bins=100)
    ax1.set_title(f'{article_type} article length distribution')
    ax1.set_xlabel('Article length')
    ax1.set_ylabel('# of articles')
    ax1.set_xlim(global_min_length, global_max_length)  # Set uniform x-axis limits

    # The second subplot shows the distribution of the log of the length of the articles to better visualize the distribution
    ax2.hist(np.log(type_df['length']), bins=100)
    ax2.set_title(f'{article_type} article length distribution (log scale)')
    ax2.set_xlabel('log(Article length)')
    ax2.set_ylabel('# of articles')
    ax2.set_xlim(global_min_log_length, global_max_log_length)  # Set uniform x-axis limits

    fig1.savefig(f'{output_dir}/{article_type}_article_length_distribution.png')
    plt.show()

In [ ]:
word_counts = {}
for content in df['content']:
    for word in content:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

In [ ]:
fig1 = plt.figure(figsize=(20, 5))
ax1 = fig1.add_subplot(111)
ax1.bar(*zip(*sorted(word_counts.items(), key=lambda x: x[1], reverse=True)[:100]))
ax1.set_title('10 most common words')
ax1.set_xlabel('Words')
plt.xticks(rotation=45)
ax1.set_ylabel('# of occurrences')
fig1.savefig('data/10_most_common_words.png')
fig1.show()